In [2]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate , PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama.chat_models import ChatOllama

import warnings
warnings.filterwarnings('ignore')

import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"



c:\Users\Sanya Kulthia\OneDrive\Desktop\sans_bot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LOAD DOCX

In [3]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

local_path = "Rag-docs.docx"

if local_path:
    loader = UnstructuredWordDocumentLoader(file_path=local_path)
    data = loader.load()
    print(f"DOCX loaded successfully: {local_path}")
else:
    print("Upload a DOCX file")


DOCX loaded successfully: Rag-docs.docx


In [8]:
import pandas as pd

df = pd.read_excel("Book1.xlsx")
df.to_csv("data_utf8.csv", index=False, encoding="utf-8")
df

,Sanskriit_Title,Sanskrit_Lines,English_Title,Author_name,Summary
0,मूर्खभृत्यस्य,"""अरे शंखनाद, गच्छापणम्, शर्कराम् आनय ।"" इति स्...",The Foolish Servant,Visnnusarman,A master orders his servant Shankhanad to brin...
1,चतुरस्य कालीदासस्य ।,"घोषितं कदाचित् भोजराज्ञा, यदि कोऽपि कविः मम दर...",Of the Clever Kalidasa,Kedar Naphade,"King Bhoja promises a reward for new poetry, b..."
2,वृद्धायाः चार्तुयम्,आसीत् चित्रपुरम् नाम किमपि नगरं श्रीपर्वतस्य स...,The cleverness of an old woman,Traditional Sanskrit story,Villagers believe a demon rings a bell on a mo...
3,शीतं बहु बाधति ।,सर्वे जानन्ति यत् भोजराज्ञः दरबारे अविद्यत कवि...,The Cold Hurts Very Much,Kedar Naphade,Kalidasa disguises himself as a palanquin carr...


In [11]:
from langchain_core.documents import Document

documents = []

for _, row in df.iterrows():
    story = row["Sanskriit_Title"]

    # Sanskrit source text
    documents.append(
        Document(
            page_content=row["Sanskrit_Lines"],
            metadata={
                "type": "sanskrit",
                "story": story,
                "title_en": row["English_Title"]
            }
        )
    )

    # Summary
    documents.append(
        Document(
            page_content=row["Summary"],
            metadata={
                "type": "summary",
                "story": story
            }
        )
    )

    # Explanation / commentary
    documents.append(
        Document(
            page_content=row["English_Title"],
            metadata={
                "type": "Title",
                "story": story
            }
        )
    )

    # Author metadata
    documents.append(
        Document(
            page_content=row["Author_name"],
            metadata={
                "type": "author",
                "story": story
            }
        )
    )

print(f"Total documents created: {len(documents)}")


Total documents created: 16


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

sanskrit_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

english_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

final_chunks = []

for doc in documents:
    if doc.metadata["type"] == "sanskrit":
        chunks = sanskrit_splitter.split_documents([doc])
    else:
        chunks = english_splitter.split_documents([doc])

    final_chunks.extend(chunks)

print(f"Total chunks after splitting: {len(final_chunks)}")


Total chunks after splitting: 41


In [13]:
for c in final_chunks[:5]:
    print("\n---")
    print(c.metadata)
    print(c.page_content[:200])



---
{'type': 'sanskrit', 'story': 'मूर्खभृत्यस्य', 'title_en': 'The Foolish Servant'}
"अरे शंखनाद, गच्छापणम्, शर्कराम् आनय ।" इति स्वभृत्यम् शंखनादम् गोवर्धनदासः आदिशति । ततः शंखनादः आपणम् गच्छति, शर्कराम् जीर्णे वस्त्रे न्यस्यति च । तस्मात् जीर्णवस्त्रात् मार्गे एव सर्वापि शर्करा स्त्

---
{'type': 'sanskrit', 'story': 'मूर्खभृत्यस्य', 'title_en': 'The Foolish Servant'}
मूढ, कुत्रास्ति शर्करा ? शर्करादिकम् एवम् जीर्णेन वस्त्रेण न एवानयन्ति कदापि । इतःपरम् किमपि वस्तुजातम् दृढायाम् सन्चिकायाम् निक्षिप्य आनय च " इति । अत्रान्तरे गोवर्धनदासस्य पुत्रः "श्वानशावकम् आनय" इ

---
{'type': 'sanskrit', 'story': 'मूर्खभृत्यस्य', 'title_en': 'The Foolish Servant'}
सन्चिकायाम् क्षिपति, सन्चिकाम् वस्त्रेण आच्छादयति च । तेन शावकस्य श्वासः रुध्दः भवति । सः च श्वानशावकः पञ्चत्वम् गच्छति । तदा गोवर्धनदासः शंखनादम् अभिधावति क्रोधेनाक्रोशति च, " धिक् मुढ, श्वानादिकम् द

---
{'type': 'sanskrit', 'story': 'मूर्खभृत्यस्य', 'title_en': 'The Foolish Servant'}
इति । ततः कदाचित् गोवर्धनदासस्य भार्या "दुग्धम् आ

In [9]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

loader = UnstructuredWordDocumentLoader("Rag-docs.docx")
docs = loader.load()

raw_text = docs[0].page_content


In [10]:
story_titles = [
    "मूर्खभृत्यस्य",
    "चतुरस्य कालीदासस्य",
    "वृद्धायाः चार्तुयम्",
    "शीतं बहु बाधति"
]

sections = []
current_story = None
buffer = []

for line in raw_text.splitlines():
    line = line.strip()

    if line in story_titles:
        if buffer and current_story:
            sections.append((current_story, "\n".join(buffer)))
            buffer = []
        current_story = line
    else:
        buffer.append(line)

# add last
if buffer and current_story:
    sections.append((current_story, "\n".join(buffer)))


In [17]:
import re

DEVANAGARI_RE = re.compile(r'[\u0900-\u097F]')
ENGLISH_RE = re.compile(r'[A-Za-z]')

def extract_sanskrit(text):
    return " ".join(
        part for part in re.split(r'[()]', text)
        if DEVANAGARI_RE.search(part) and not ENGLISH_RE.search(part)
    )

def extract_english(text):
    return " ".join(
        part for part in re.split(r'[()]', text)
        if ENGLISH_RE.search(part)
    )


In [18]:
from langchain_core.documents import Document

structured_docs = []

for story, text in sections:
    sanskrit_parts = []
    english_parts = []

    for line in text.splitlines():
        s_part = extract_sanskrit(line)
        e_part = extract_english(line)

        if s_part.strip():
            sanskrit_parts.append(s_part.strip())
        if e_part.strip():
            english_parts.append(e_part.strip())

    if sanskrit_parts:
        structured_docs.append(
            Document(
                page_content="\n".join(sanskrit_parts),
                metadata={"type": "sanskrit", "story": story}
            )
        )

    if english_parts:
        structured_docs.append(
            Document(
                page_content="\n".join(english_parts),
                metadata={"type": "english_summary", "story": story}
            )
        )


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


sanskrit_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

english_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

In [20]:
final_chunks = []

for doc in structured_docs:
    if doc.metadata["type"] == "sanskrit":
        chunks = sanskrit_splitter.split_documents([doc])
    else:
        chunks = english_splitter.split_documents([doc])

    final_chunks.extend(chunks)


In [21]:
from collections import Counter

type_counts = Counter([doc.metadata["type"] for doc in final_chunks])
story_counts = Counter([doc.metadata["story"] for doc in final_chunks])

print("Chunk count by type:")
print(type_counts)

print("\nChunk count by story:")
print(story_counts)


Chunk count by type:
Counter({'sanskrit': 25, 'english_summary': 11})

Chunk count by story:
Counter({'वृद्धायाः चार्तुयम्': 20, 'मूर्खभृत्यस्य': 16})


In [22]:
def preview_chunks(chunks, n=2):
    for doc_type in ["sanskrit", "english_summary"]:
        print(f"\n===== SAMPLE {doc_type.upper()} CHUNKS =====")
        shown = 0
        for doc in chunks:
            if doc.metadata["type"] == doc_type:
                print("\n--- Story:", doc.metadata["story"], "---")
                print(doc.page_content[:500])
                shown += 1
                if shown >= n:
                    break

preview_chunks(final_chunks)



===== SAMPLE SANSKRIT CHUNKS =====

--- Story: मूर्खभृत्यस्य ---
"अरे शंखनाद, गच्छापणम्, शर्कराम् आनय ।" इति स्वभृत्यम् शंखनादम् गोवर्धनदासः आदिशति । ततः शंखनादः आपणम् गच्छति, शर्कराम् जीर्णे वस्त्रे न्यस्यति च । तस्मात् जीर्णवस्त्रात् मार्गे एव सर्वापि शर्करा स्त्रवति । ततः गोवर्धनदासः कोपेन शंखनादम् वदति, "अरे मूढ, कुत्रास्ति शर्करा ? शर्करादिकम् एवम् जीर्णेन

--- Story: मूर्खभृत्यस्य ---
मूढ, कुत्रास्ति शर्करा ? शर्करादिकम् एवम् जीर्णेन वस्त्रेण न एवानयन्ति कदापि । इतःपरम् किमपि वस्तुजातम् दृढायाम् सन्चिकायाम् निक्षिप्य आनय च " इति । अत्रान्तरे गोवर्धनदासस्य पुत्रः "श्वानशावकम् आनय" इति शंखनादम् आदिशति । आज्ञापालकः शंखनादः श्वानशावकम् सन्चिकायाम् क्षिपति, सन्चिकाम् वस्त्रेण

===== SAMPLE ENGLISH_SUMMARY CHUNKS =====

--- Story: मूर्खभृत्यस्य ---
चतुरस्य कालीदासस्य । Of the clever kAlIdAsa by: Kedar Naphade,                                       ksn2@lehigh.edu

--- Story: मूर्खभृत्यस्य ---
King Bhoj once proclaimed, "If any poet reads a new poem in my court, I shall give him One La

In [1]:
from docx import Document
import regex as re


In [2]:
DEVANAGARI = re.compile(r'[\p{Devanagari}]')
LATIN = re.compile(r'[A-Za-z]')

def detect_script_and_language(text: str):
    has_dev = bool(DEVANAGARI.search(text))
    has_lat = bool(LATIN.search(text))

    if has_dev and has_lat:
        return "mixed", "mixed"
    elif has_dev:
        return "devanagari", "sa"
    elif has_lat:
        return "latin", "en"
    else:
        return "unknown", "unknown"


In [3]:
def load_docx(path):
    doc = Document(path)
    return [p.text.strip() for p in doc.paragraphs if p.text.strip()]

paragraphs = load_docx("Rag-docs.docx")
len(paragraphs)


39

In [4]:
def build_chunks(paragraphs, document_id="rag_docs_001"):
    chunks = []
    section_id = 0

    for idx, para in enumerate(paragraphs):

        # heuristic section break
        if len(para) < 40:
            section_id += 1

        script, language = detect_script_and_language(para)

        chunk = {
            "document_id": document_id,
            "section_id": section_id,
            "chunk_id": f"{document_id}_{idx}",
            "text": para,
            "language": language,
            "script": script,
            "position": idx
        }

        chunks.append(chunk)

    return chunks

chunks = build_chunks(paragraphs)
len(chunks)


39

In [6]:
import json

with open("chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("Saved", len(chunks), "chunks to chunks.json")


Saved 39 chunks to chunks.json


In [5]:
for c in chunks[:5]:
    print("-" * 80)
    print("Chunk ID :", c["chunk_id"])
    print("Language :", c["language"])
    print("Script   :", c["script"])
    print("Text     :", c["text"][:300])


--------------------------------------------------------------------------------
Chunk ID : rag_docs_001_0
Language : sa
Script   : devanagari
Text     : मूर्खभृत्यस्य
--------------------------------------------------------------------------------
Chunk ID : rag_docs_001_1
Language : sa
Script   : devanagari
Text     : "अरे शंखनाद, गच्छापणम्, शर्कराम् आनय ।" इति स्वभृत्यम् शंखनादम् गोवर्धनदासः आदिशति । ततः शंखनादः आपणम् गच्छति, शर्कराम् जीर्णे वस्त्रे न्यस्यति च । तस्मात् जीर्णवस्त्रात् मार्गे एव सर्वापि शर्करा स्त्रवति । ततः गोवर्धनदासः कोपेन शंखनादम् वदति, "अरे मूढ, कुत्रास्ति शर्करा ? शर्करादिकम् एवम् जीर्णेन व
--------------------------------------------------------------------------------
Chunk ID : rag_docs_001_2
Language : sa
Script   : devanagari
Text     : वरम् भृत्यविहिनस्य जिवितम् श्रमपूरितम् । मूर्खभृत्यस्य संसर्गात् सर्वम् कार्यम् विनश्यति ॥
--------------------------------------------------------------------------------
Chunk ID : rag_docs_001_3
Language : mixed
Script  